In [2]:
import json
import glob
import os

In [3]:
config_filename = "multi_config_v1.json"

config = dict()

model_config = dict()
model_config["name"] = "DynUNet"  # network model name from MONAI
# set the network hyper-parameters
model_config["in_channels"] = 4  # 4 input images for the BraTS challenge
model_config["out_channels"] = 1   # whole tumor, tumor core, enhancing tumor
model_config["spatial_dims"] = 3   # 3D input images
model_config["deep_supervision"] = False  # do not check outputs of lower layers
model_config["strides"] = [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]][:-1]  # number of downsampling convolutions
model_config["filters"] = [64, 96, 128, 192, 256, 384, 512, 768, 1024][:len(model_config["strides"])]  # number of filters per layer
model_config["kernel_size"] = [[3, 3, 3]] * len(model_config["strides"])  # size of the convolution kernels per layer
model_config["upsample_kernel_size"] = model_config["strides"][1:]  # should be the same as the strides

# put the model config in the main config
config["model"] = model_config

config["optimizer"] = {'name': 'Adam', 
                       'lr': 0.001}  # initial learning rate

# define the loss
config["loss"] = {'name': 'DiceLoss', # from Monai
                  'include_background': True,  # we do not have a label for the background, so this should be true (by "include background" monai means include channel 0)
                  'sigmoid': True,
                  'batch': True}  # transform the model logits to activations

# set the cross validation parameters
config["cross_validation"] = {'folds': 5,  # number of cross validation folds
                              'seed': 25},  # seed to make the generation of cross validation folds consistent across different trials
# set the scheduler parameters
config["scheduler"] = {'name': 'ReduceLROnPlateau', 
                       'patience': 5,  # wait 5 epochs with no improvement before reducing the learning rate
                       'factor': 0.5,   # multiply the learning rate by 0.5
                       'min_lr': 1e-08}  # stop reducing the learning rate once it gets to 1e-8

# set the dataset parameters
roi_size = [192, 192, 96]
config["dataset"] = {'name': 'SegmentationDatasetPersistent',  # 'Persistent' means that it will save the preprocessed outputs generated during the first epoch
# However, using 'Persistent', does also increase the time of the first epoch compared to the other epochs, which should run faster
  'labels': [1],  # 1: tumor 
  'normalization': 'ScaleIntensityD',  # scale intensity from 0 to 1
  'normalization_kwargs': {'channel_wise': True},  # perform the normalization channel wise
  'orientation': 'RAS',  # Force all the images to be the same orientation (Right-Anterior-Suppine)
  'training':  # the following arguments will only be applied to the training data.
    {
    'desired_shape': roi_size,  # resize the images to this shape, increase this to get higher resolution images (increases computation time and memory usage)
    'random_crop': True,  # resample the images when resizing them, otherwise the resize could crop out regions of interest
    'crop_foreground': True,  # crop the foreground of the images
    'foreground_percentile': 0.75,  # aggressive foreground cropping to make sure the empty space is taken out of the images
    'spatial_augmentations': [{'name': 'RandFlipD', 'spatial_axis': 0, 'prob': 0.5},
                              {'name': 'RandFlipD', 'spatial_axis': 1, 'prob': 0.5}],
    'intensity_augmentations': [{'name': 'RandScaleIntensityD', 'factors': 0.1, 'prob': 1.0},
                                {'name': 'RandShiftIntensityD', 'offsets': 0.1, 'prob': 1.0}],
    }
}

config["inference"] = {'name': 'SlidingWindowInferer',
                       'roi_size': roi_size,
                       'mode': 'gaussian'}  # sliding window inference for validation

config["training"] = {'batch_size': 4,  # number of image/label pairs to read at a time during training
  'validation_batch_size': 1,  # number of image/label pairs to read at atime during validation
  'amp': False,  # don't set this to true unless the model you are using is setup to use automatic mixed precision (AMP)
  'early_stopping_patience': None,  # stop the model early if the validaiton loss stops improving
  'n_epochs': 100,  # number of training epochs, reduce this if you don't want training to run as long
  'save_every_n_epochs': None,  # save the model every n epochs (otherwise only the latest model will be saved)
  'save_last_n_models': None,  # save the last n models 
  'save_best': True, # save the model that has the best validation loss
  'training_iterations_per_epoch': 10}  # validation takes a long time, so I don't want to validate every training iteration

In [4]:
# get the training filenames
config["training_filenames"] = list()
ground_truth_filenames = sorted(glob.glob("./aligned/*/*/*NB*.nii*"))
for label_filename in ground_truth_filenames:
    subject, visit = label_filename.split("/")[-3:-1]
    filenames = sorted(glob.glob(os.path.join("aligned", subject, visit, f"*.nii*")))
    n_features = len(filenames) 
    feature_modalities = ["_".join(fn.split("_")[3:-1]).strip("8 ") for fn in filenames]
    t1_filename = filenames[feature_modalities.index("T1_gd")]
    assert os.path.exists(t1_filename)
    
    if len(feature_modalities) < 5:
        continue
    
    if "T2" not in feature_modalities:
        for filename in filenames:
            if "T2" in filename:
                t2_fn = filename
        print(t2_fn)
    else:
        t2_fn = filenames[feature_modalities.index("T2")]
    assert os.path.exists(t2_fn)
    
    if "DWI_b0" not in feature_modalities:
        for filename in filenames:
            if "DWI" in filename and "b0" in filename:
                dwi_b0 = filename
        print(dwi_b0)
    else:
        dwi_b0 = filenames[feature_modalities.index("DWI_b0")]
    assert os.path.exists(dwi_b0)

    if "DWI_b100" not in feature_modalities:
        for filename in filenames:
            if "DWI" in filename and "b100" in filename:
                dwi_b100 = filename
        print(dwi_b100)
    else:
        dwi_b100 = filenames[feature_modalities.index("DWI_b100")]
    assert os.path.exists(dwi_b100)

    
    config["training_filenames"].append({"image": [t1_filename, t2_fn, dwi_b0, dwi_b100], "label": label_filename})

aligned/PT_77/20210507/PT_77_T2)29210507.nii.gz
aligned/PT_80/20190410/PT_80_DWI_b0.nii.gz
aligned/PT_80/20190410/PT_80_DWI_b100.nii.gz
aligned/PT_81/20200129/PT_81_T2_20200129.nii.gz


In [5]:
len(config["training_filenames"])

72

In [61]:
with open(config_filename, "w") as op:
    json.dump(config, op, indent=4)

In [51]:
filenames

['aligned/PT_80/20190410/PT_80_DWI_b0.nii.gz',
 'aligned/PT_80/20190410/PT_80_DWI_b100.nii.gz',
 'aligned/PT_80/20190410/PT_80_NB_20190410.nii.gz',
 'aligned/PT_80/20190410/PT_80_T1_gd_20190410.nii',
 'aligned/PT_80/20190410/PT_80_T2_20190410.nii.gz']

In [50]:
feature_modalities

['DWI', 'DWI', 'NB', 'T1_gd', 'T2']

In [28]:
import torch

In [14]:
import numpy as np
a = np.asarray([1, np.nan, 2, np.nan, 3])

In [17]:
a.min()

nan

In [6]:
torch.quantile(torch.rand(100), 1.1)

RuntimeError: quantile() q must be in the range [0, 1] but got 1.1

In [29]:
import nibabel as nib
import numpy as np
image = nib.load("./T1w_config_v2/fold2/validation/PT_16_T1_gd_20220712.nii")

In [30]:
image.shape

(384, 384, 280)

In [31]:
data = np.asarray(image.dataobj)

In [36]:
gt_image = nib.load("./train/PT_16/20220712/PT_16_NB_20220712.nii.gz")

In [37]:
gt_data = np.asarray(gt_image.dataobj)

In [38]:
np.unique(gt_data)

array([0, 1], dtype=int16)

In [39]:
gt_data = np.asarray(gt_data > 0, np.int16)

In [40]:
from monai.losses.dice import DiceLoss

In [62]:
crit = DiceLoss(sigmoid=True, batch=False, include_background=True)

In [63]:
crit(torch.from_numpy(data)[None, None], torch.from_numpy(gt_data)[None, None])

tensor(0.9633)

In [11]:
data.mean()

84.63820795679383

In [12]:
import torch

In [27]:
np.percentile(torch.flatten(torch.from_numpy(np.asarray(data, float)), start_dim=-3), 75, axis=-1)

119.0

In [21]:
torch.kthvalue?

Docstring:
kthvalue(input, k, dim=None, keepdim=False, *, out=None) -> (Tensor, LongTensor)

Returns a namedtuple ``(values, indices)`` where ``values`` is the :attr:`k` th
smallest element of each row of the :attr:`input` tensor in the given dimension
:attr:`dim`. And ``indices`` is the index location of each element found.

If :attr:`dim` is not given, the last dimension of the `input` is chosen.

If :attr:`keepdim` is ``True``, both the :attr:`values` and :attr:`indices` tensors
are the same size as :attr:`input`, except in the dimension :attr:`dim` where
they are of size 1. Otherwise, :attr:`dim` is squeezed
(see :func:`torch.squeeze`), resulting in both the :attr:`values` and
:attr:`indices` tensors having 1 fewer dimension than the :attr:`input` tensor.

.. note::
    When :attr:`input` is a CUDA tensor and there are multiple valid
    :attr:`k` th values, this function may nondeterministically return
    :attr:`indices` for any of them.

Args:
    input (Tensor): the input tenso